# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,59.52,51,78,11.45,AU,1697168463
1,1,khanapur,15.6333,74.5167,78.42,59,42,8.75,IN,1697168463
2,2,port elizabeth,-33.9180,25.5701,59.29,95,40,33.38,ZA,1697168463
3,3,grytviken,-54.2811,-36.5092,26.17,83,95,5.79,GS,1697168464
4,4,dinskaya,45.2178,39.2283,48.90,75,2,3.31,RU,1697168464


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    size = "Humidity"
)

# Display the map
map_plot


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[city_data_df["Max Temp"]< 80]
city_data_df = city_data_df.loc[city_data_df["Max Temp"]> 70]
city_data_df = city_data_df.loc[city_data_df["Cloudiness"]<20]
city_data_df = city_data_df.loc[city_data_df["Humidity"]<70]

# Drop any rows with null values
city_data_df.dropna(axis=0, how="any")

# Display sample data
city_data_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,ishinomaki,38.4167,141.3000,73.49,48,1,12.06,JP,1697168469
30,30,turabah,21.2141,41.6331,73.49,20,0,4.32,SA,1697168473
56,56,hondarribia,43.3720,-1.7945,77.65,41,0,5.75,ES,1697168481
82,82,lake havasu city,34.4839,-114.3224,75.81,17,0,16.11,US,1697168490
117,117,ad dilam,23.9915,47.1627,77.41,18,0,8.72,SA,1697168501
139,139,namie,37.4833,141.0000,72.72,43,0,4.38,JP,1697168507
144,144,margaret river,-33.9500,115.0667,71.44,46,0,7.45,AU,1697168509
160,160,crane,31.3974,-102.3501,76.73,28,0,7.00,US,1697168513
167,167,surab,28.4914,66.2585,73.42,15,0,6.31,PK,1697168516
229,229,safford,32.8340,-109.7076,72.52,30,0,16.11,US,1697168536


### Step 3: Create a new DataFrame called `hotel_df`.

In [4]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
20,ishinomaki,JP,38.4167,141.3000,48,
30,turabah,SA,21.2141,41.6331,20,
56,hondarribia,ES,43.3720,-1.7945,41,
82,lake havasu city,US,34.4839,-114.3224,17,
117,ad dilam,SA,23.9915,47.1627,18,
139,namie,JP,37.4833,141.0000,43,
144,margaret river,AU,-33.9500,115.0667,46,
160,crane,US,31.3974,-102.3501,28,
167,surab,PK,28.4914,66.2585,15,
229,safford,US,32.8340,-109.7076,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "limit": 1,
    "bias": "",
    "lang": "en",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

 
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
ishinomaki - nearest hotel: 丹美旅館
turabah - nearest hotel: No hotel found
hondarribia - nearest hotel: Hotel Jauregui
lake havasu city - nearest hotel: No hotel found
ad dilam - nearest hotel: No hotel found
namie - nearest hotel: HOTELなみえ
margaret river - nearest hotel: Margaret River Hotel
crane - nearest hotel: No hotel found
surab - nearest hotel: No hotel found
safford - nearest hotel: Quality Inn And Suites
gunnedah - nearest hotel: The Court House
alice springs - nearest hotel: Aurora Alice Springs
kasungu - nearest hotel: Kasungu Inn
sydney - nearest hotel: The Westin
iwaki - nearest hotel: Iwaki Prince Hotel
toliara - nearest hotel: Ambary
aoulef - nearest hotel: دار الضياف
big spring - nearest hotel: Settles Hotel
paramirim - nearest hotel: No hotel found
urangan - nearest hotel: Ramada
isla vista - nearest hotel: The Club & Guest House
al kharijah - nearest hotel: Kharga Hotel
hasaki - nearest hotel: 大新旅館
yuma - nearest hotel: Hilton Garden Inn Yuma Pivo

,City,Country,Lat,Lng,Humidity,Hotel Name
20,ishinomaki,JP,38.4167,141.3000,48,丹美旅館
30,turabah,SA,21.2141,41.6331,20,No hotel found
56,hondarribia,ES,43.3720,-1.7945,41,Hotel Jauregui
82,lake havasu city,US,34.4839,-114.3224,17,No hotel found
117,ad dilam,SA,23.9915,47.1627,18,No hotel found
139,namie,JP,37.4833,141.0000,43,HOTELなみえ
144,margaret river,AU,-33.9500,115.0667,46,Margaret River Hotel
160,crane,US,31.3974,-102.3501,28,No hotel found
167,surab,PK,28.4914,66.2585,15,No hotel found
229,safford,US,32.8340,-109.7076,30,Quality Inn And Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE